In [1]:
import pandas as pd
import os
import requests
import sqlite3
import json
import time

In [2]:
API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"

# Core endpoints for reference data
CORE_BASE = "https://api.sportmonks.com/v3/core"

# SQLite database path
DB_PATH = "sportmonks.db"


In [3]:
def fetch_all_core(endpoint):
    """
    Paginate through CORE_BASE/{endpoint} with per_page=1000
    Returns a list of all items.
    """
    all_items = []
    page = 1
    while True:
        resp = requests.get(
            f"{CORE_BASE}/{endpoint}",
            params={"api_token": API_TOKEN, "per_page": 1000, "page": page}
        )
        resp.raise_for_status()
        data = resp.json().get("data", [])
        if not data:
            break
        all_items.extend(data)
        page += 1
    return all_items

In [4]:
import sqlite3

DB_PATH = "sportmonks.db"

# Use timeout to wait up to 30 seconds for any locks to clear
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    
    # Drop old tables
    for tbl in ["cities", "regions", "countries", "continents"]:
        cur.execute(f"DROP TABLE IF EXISTS {tbl}")
    
    # Re-create tables
    cur.execute("""
    CREATE TABLE continents (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        code TEXT
    );
    """)
    cur.execute("""
    CREATE TABLE countries (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        code TEXT,
        continent_id INTEGER,
        FOREIGN KEY(continent_id) REFERENCES continents(id)
    );
    """)
    cur.execute("""
    CREATE TABLE regions (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        country_id INTEGER NOT NULL,
        FOREIGN KEY(country_id) REFERENCES countries(id)
    );
    """)
    cur.execute("""
    CREATE TABLE cities (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        region_id INTEGER,
        country_id INTEGER NOT NULL,
        FOREIGN KEY(region_id) REFERENCES regions(id),
        FOREIGN KEY(country_id) REFERENCES countries(id)
    );
    """)
    # commit happens automatically on exit of the with-block

# Verify
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cur.fetchall()]
    print("Tables now in DB:", tables)


Tables now in DB: ['leagues', 'seasons', 'fixtures', 'players', 'continents', 'countries', 'regions', 'cities']


In [4]:
import sqlite3

DB_PATH = "sportmonks.db"

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    
    # Continents
    for c in fetch_all_core("continents"):
        cur.execute(
            "INSERT OR REPLACE INTO continents (id, name, code) VALUES (?,?,?)",
            (c["id"], c["name"], c.get("code"))
        )
    
    # Countries
    for c in fetch_all_core("countries"):
        cur.execute(
            "INSERT OR REPLACE INTO countries (id, name, code, continent_id) VALUES (?,?,?,?)",
            (c["id"], c["name"], c.get("code"), c.get("continent_id"))
        )
    
    # Regions
    for r in fetch_all_core("regions"):
        cur.execute(
            "INSERT OR REPLACE INTO regions (id, name, country_id) VALUES (?,?,?)",
            (r["id"], r["name"], r["country_id"])
        )
    
        # Cities: fetch via venues?include=city to avoid rate-limit
    FB_BASE = "https://api.sportmonks.com/v3/football"
    page = 1
    unique_cities = {}
    while True:
        resp = requests.get(
            f"{FB_BASE}/venues",
            params={
                "api_token": API_TOKEN,
                "include": "city",
                "per_page": 1000,
                "page": page
            }
        )
        resp.raise_for_status()
        venues = resp.json().get("data", [])
        if not venues:
            break
        for v in venues:
            city = v.get("city")
            if city:
                unique_cities[city["id"]] = city
        page += 1

    for city in unique_cities.values():
        cur.execute(
            "INSERT OR REPLACE INTO cities (id, name, region_id, country_id) VALUES (?,?,?,?)",
            (city["id"], city["name"], city.get("region_id"), city["country_id"])
        )

    
    conn.commit()

In [5]:
def fetch_all_football(endpoint):
    items, page = [], 1
    while True:
        resp = requests.get(
            f"{FB_BASE}/{endpoint}",
            params={"api_token": API_TOKEN, "per_page": 1000, "page": page}
        )
        if resp.status_code == 429:
            # rate-limit backoff
            reset = resp.json().get("rate_limit", {}).get("resets_in_seconds", 60)
            print(f"Rate‐limit on {endpoint}, sleeping {reset+1}s…")
            time.sleep(reset + 1)
            continue
        resp.raise_for_status()
        js   = resp.json()
        data = js.get("data", [])
        if not data:
            break
        items.extend(data)
        if not js.get("pagination", {}).get("has_more", False):
            break
        page += 1
    return items

In [6]:
# 2. Create 'leagues' and 'seasons' tables
DB_PATH = "sportmonks.db"

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    # Clean slate
    cur.execute("DROP TABLE IF EXISTS seasons")
    cur.execute("DROP TABLE IF EXISTS leagues")
    # Leagues
    cur.execute("""
    CREATE TABLE leagues (
        id          INTEGER PRIMARY KEY,
        name        TEXT NOT NULL,
        country_id  INTEGER,
        logo_path   TEXT,
        slug        TEXT,
        type        TEXT,
        FOREIGN KEY(country_id) REFERENCES countries(id)
    );
    """)
    # Seasons
    cur.execute("""
    CREATE TABLE seasons (
        id         INTEGER PRIMARY KEY,
        name       TEXT,
        league_id  INTEGER,
        start_date TEXT,
        end_date   TEXT,
        FOREIGN KEY(league_id) REFERENCES leagues(id)
    );
    """)
    conn.commit()

In [7]:
# 3. Fetch & insert all leagues
leagues = fetch_all_football("leagues")
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    for l in leagues:
        cur.execute(
            "INSERT OR REPLACE INTO leagues (id,name,country_id,logo_path,slug,type) VALUES (?,?,?,?,?,?)",
            (
                l["id"],
                l["name"],
                l.get("country_id"),
                l.get("logo_path"),
                l.get("slug"),
                l.get("type")
            )
        )
    conn.commit()
print(f"✔ Loaded {len(leagues)} leagues")

✔ Loaded 27 leagues


In [8]:
# 4. Fetch & insert all seasons
seasons = fetch_all_football("seasons")
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    for s in seasons:
        cur.execute(
            "INSERT OR REPLACE INTO seasons (id,name,league_id,start_date,end_date) VALUES (?,?,?,?,?)",
            (
                s["id"],
                s["name"],
                s.get("league_id"),
                s.get("start_date"),
                s.get("end_date")
            )
        )
    conn.commit()
print(f"✔ Loaded {len(seasons)} seasons")

✔ Loaded 531 seasons


In [10]:
# 5. Verify counts
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM leagues");  print("Leagues: ", cur.fetchone()[0])
    cur.execute("SELECT COUNT(*) FROM seasons");  print("Seasons:", cur.fetchone()[0])

Leagues:  27
Seasons: 531


In [12]:
# ── 5. Fetch ALL fixtures, then keep only last-5 seasons ─────────────────────
import sqlite3
import requests
import time

FB_BASE = "https://api.sportmonks.com/v3/football"
DB_PATH = "sportmonks.db"

# 5a) Build set of last-5‐season IDs per league
keep_seasons = set()
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute("SELECT id FROM leagues;")
    for (lid,) in cur.fetchall():
        cur.execute("""
          SELECT id
            FROM seasons
           WHERE league_id=?
        ORDER BY start_date DESC
           LIMIT 5
        """, (lid,))
        keep_seasons |= {r[0] for r in cur.fetchall()}
print(f"→ Keeping {len(keep_seasons)} season IDs")

# 5b) Drop & recreate fixtures table
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS fixtures;")
    cur.execute("""
    CREATE TABLE fixtures (
        id            INTEGER PRIMARY KEY,
        season_id     INTEGER,
        league_id     INTEGER,
        date_time     TEXT,
        status        TEXT,
        home_team_id  INTEGER,
        away_team_id  INTEGER,
        home_score    INTEGER,
        away_score    INTEGER,
        FOREIGN KEY(season_id) REFERENCES seasons(id),
        FOREIGN KEY(league_id) REFERENCES leagues(id)
    );
    """)
    conn.commit()

# 5c) Page through ALL fixtures endpoint and insert only desired seasons
page = 1
total = 0
while True:
    resp = requests.get(
        f"{FB_BASE}/fixtures",
        params={"api_token": API_TOKEN, "per_page": 1000, "page": page},
        timeout=10
    )
    # rate-limit back-off
    if resp.status_code == 429:
        wait = resp.json().get("rate_limit", {}).get("resets_in_seconds", 60)
        print(f"Rate-limit, sleeping {wait+1}s…")
        time.sleep(wait + 1)
        continue
    resp.raise_for_status()

    data = resp.json().get("data", [])
    if not data:
        break

    # Insert only fixtures from seasons we care about
    with sqlite3.connect(DB_PATH, timeout=30) as conn:
        cur = conn.cursor()
        for f in data:
            sid = f.get("season_id")
            if sid not in keep_seasons:
                continue
            cur.execute("""
              INSERT OR REPLACE INTO fixtures
                (id,season_id,league_id,date_time,status,
                 home_team_id,away_team_id,home_score,away_score)
              VALUES (?,?,?,?,?,?,?,?,?);
            """, (
                f["id"], sid, f.get("league_id"),
                f.get("starting_at"), f.get("time", {}).get("status"),
                f.get("localteam_id"), f.get("visitorteam_id"),
                f.get("scores", {}).get("localteam_score"),
                f.get("scores", {}).get("visitorteam_score")
            ))
            total += 1
        conn.commit()

    print(f"Page {page}: fetched {len(data)} fixtures, kept {total} so far")
    page += 1
    time.sleep(0.2)

print(f"✔ Fixtures total (last 5 seasons): {total}")


→ Keeping 133 season IDs
Rate-limit, sleeping 61s…
Rate-limit, sleeping 61s…


KeyboardInterrupt: 

In [59]:
resp = requests.get("https://api.sportmonks.com/v3/football/teams", params={"api_token": API_TOKEN, "per_page":1})
print("Status code:", resp.status_code)
print("Sample:", resp.json().get("data", []))

Status code: 200
Sample: [{'id': 1, 'sport_id': 1, 'country_id': 462, 'venue_id': 214, 'gender': 'male', 'name': 'West Ham United', 'short_code': 'WHU', 'image_path': 'https://cdn.sportmonks.com/images/soccer/teams/1/1.png', 'founded': 1895, 'type': 'domestic', 'placeholder': False, 'last_played_at': '2025-04-26 14:00:00'}]


In [60]:
CACHE_FILE = "players.json"

def fetch_all_football_with_cache(endpoint, cache_file):
    if os.path.exists(cache_file):
        print(f"Loading players from cache: {cache_file}")
        return json.load(open(cache_file))
    print("Fetching players from API…")
    items, page = [], 1
    while True:
        params = {"api_token": API_TOKEN, "per_page": 1000, "page": page}
        for attempt in range(3):
            try:
                resp = requests.get(f"{FB_BASE}/{endpoint}", params=params, timeout=10)
                if resp.status_code == 429:
                    reset = resp.json().get("rate_limit",{}).get("resets_in_seconds",60)
                    print(f"Rate‐limit, sleeping {reset+1}s…")
                    time.sleep(reset+1)
                    continue
                resp.raise_for_status()
                break
            except Exception as e:
                wait = 2 ** attempt
                print(f"Request failed ({e}), retrying in {wait}s…")
                time.sleep(wait)
        else:
            raise RuntimeError("Failed after 3 retries")
        js = resp.json()
        data = js.get("data", [])
        if not data:
            break
        items.extend(data)
        if not js.get("pagination", {}).get("has_more", False):
            break
        page += 1
    # Cache to disk
    with open(cache_file, "w") as f:
        json.dump(items, f)
    return items

In [34]:
# ── Create & Load Players ─────────────────────────────────────────────────────
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS players;")
    cur.execute("""
    CREATE TABLE players (
        id            INTEGER PRIMARY KEY,
        firstname     TEXT,
        lastname      TEXT,
        fullname      TEXT,
        common_name   TEXT,
        nationality   TEXT,
        country_id    INTEGER,
        birthcountry  TEXT,
        birthdate     TEXT,
        position      TEXT,
        height        INTEGER,
        weight        INTEGER,
        image_path    TEXT,
        team_id       INTEGER
    );""")
    conn.commit()

players = fetch_all_football_with_cache("players", CACHE_FILE)

with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    for p in players:
        cur.execute(
            "INSERT OR REPLACE INTO players "
            "(id,firstname,lastname,fullname,common_name,nationality,country_id,"
            "birthcountry,birthdate,position,height,weight,image_path,team_id) "
            "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?);",
            (
                p["id"], p.get("firstname"), p.get("lastname"), p.get("fullname"),
                p.get("common_name"), p.get("nationality"), p.get("country_id"),
                p.get("birthcountry"), p.get("birthdate"), p.get("position"),
                p.get("height"), p.get("weight"), p.get("image_path"), p.get("team_id")
            )
        )
    conn.commit()

print(f"✔ Loaded {len(players)} players")

Fetching players from API…
Request failed (HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)), retrying in 1s…
Request failed (HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)), retrying in 1s…
Request failed (HTTPSConnectionPool(host='api.sportmonks.com', port=443): Read timed out. (read timeout=10)), retrying in 1s…
Rate‐limit, sleeping 61s…
Rate‐limit, sleeping 61s…
Rate‐limit, sleeping 61s…


RuntimeError: Failed after 3 retries

In [37]:
import sqlite3
import requests
import time

API_TOKEN  = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
FB_BASE    = "https://api.sportmonks.com/v3/football"
DB_PATH    = "sportmonks.db"

def fetch_all_football(endpoint, extra_params=None):
    items, page = [], 1
    while True:
        params = {"api_token": API_TOKEN, "per_page": 1000, "page": page}
        if extra_params:
            params.update(extra_params)
        resp = requests.get(f"{FB_BASE}/{endpoint}", params=params, timeout=10)
        if resp.status_code == 429:
            reset = resp.json().get("rate_limit", {}).get("resets_in_seconds", 60)
            time.sleep(reset + 1)
            continue
        resp.raise_for_status()
        js   = resp.json()
        data = js.get("data", [])
        if not data:
            break
        items.extend(data)
        if not js.get("pagination", {}).get("has_more", False):
            break
        page += 1
    return items

# 1. Read all country IDs
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
cur.execute("SELECT id FROM countries")
country_ids = [r[0] for r in cur.fetchall()]
conn.close()

# 2. Fetch players per country (correct endpoint path) :contentReference[oaicite:0]{index=0}
all_players = []
for cid in country_ids:
    endpoint = f"players/countries/{cid}"
    batch = fetch_all_football(endpoint)
    print(f"→ Fetched {len(batch)} players from country {cid}")
    all_players.extend(batch)

# 3. Deduplicate
unique_players = {p["id"]: p for p in all_players}.values()
players = list(unique_players)
print(f"Total unique players: {len(players)}")

# 4. Recreate & populate `players` table
with sqlite3.connect(DB_PATH, timeout=30) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS players;")
    cur.execute("""
    CREATE TABLE players (
      id            INTEGER PRIMARY KEY,
      firstname     TEXT, lastname TEXT, fullname TEXT,
      common_name   TEXT, nationality TEXT,
      country_id    INTEGER, birthcountry TEXT,
      birthdate     TEXT, position TEXT,
      height        INTEGER, weight INTEGER,
      image_path    TEXT, team_id INTEGER
    );
    """)
    for p in players:
        cur.execute(
            "INSERT OR REPLACE INTO players "
            "(id,firstname,lastname,fullname,common_name,nationality,"
            "country_id,birthcountry,birthdate,position,height,weight,"
            "image_path,team_id) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?);",
            (
              p["id"], p.get("firstname"), p.get("lastname"), p.get("fullname"),
              p.get("common_name"), p.get("nationality"), p.get("country_id"),
              p.get("birthcountry"), p.get("birthdate"), p.get("position"),
              p.get("height"), p.get("weight"), p.get("image_path"), p.get("team_id")
            )
        )
    conn.commit()

print(f"✔ Loaded {len(players)} players")

→ Fetched 2019 players from country 2
→ Fetched 2539 players from country 5
→ Fetched 2376 players from country 11
→ Fetched 3401 players from country 17
→ Fetched 1462 players from country 20
→ Fetched 319 players from country 23
→ Fetched 157 players from country 26
→ Fetched 10740 players from country 32
→ Fetched 2101 players from country 38
→ Fetched 0 players from country 41
→ Fetched 1012 players from country 44
→ Fetched 1844 players from country 47
→ Fetched 973 players from country 62
→ Fetched 105 players from country 80
→ Fetched 1757 players from country 86
→ Fetched 278 players from country 98
→ Fetched 34 players from country 107
→ Fetched 41 players from country 116
→ Fetched 176 players from country 119
→ Fetched 130 players from country 122
→ Fetched 233 players from country 125
→ Fetched 1101 players from country 143
→ Fetched 111 players from country 146
→ Fetched 0 players from country 147
→ Fetched 282 players from country 155
→ Fetched 342 players from country 15

KeyboardInterrupt: 

In [11]:
import sqlite3, requests, time
from requests.exceptions import HTTPError

API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
FB_BASE   = "https://api.sportmonks.com/v3/football"
DB = "sportmonks.db"

# 1) Gather the last 5 seasons for each league
season_ids = []
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    cur.execute("SELECT id FROM leagues")
    for (lid,) in cur.fetchall():
        cur.execute("""
            SELECT id 
              FROM seasons 
             WHERE league_id=? 
          ORDER BY start_date DESC 
             LIMIT 5
        """, (lid,))
        season_ids += [r[0] for r in cur.fetchall()]
season_ids = list(set(season_ids))
print(f"→ Fetching fixtures for {len(season_ids)} seasons (~{len(season_ids)} API calls)")

# 2) Recreate fixtures table
with sqlite3.connect(DB, timeout=30) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS fixtures;")
    cur.execute("""
      CREATE TABLE fixtures (
        id            INTEGER PRIMARY KEY,
        season_id     INTEGER,
        league_id     INTEGER,
        date_time     TEXT,
        status        TEXT,
        home_team_id  INTEGER,
        away_team_id  INTEGER,
        home_score    INTEGER,
        away_score    INTEGER
      );
    """)
    conn.commit()

# 3) Fetch + insert all fixtures
all_fx = []
for sid in season_ids:
    try:
        r = requests.get(
            f"{FB_BASE}/seasons/{sid}",
            params={"api_token": API_TOKEN, "include": "fixtures"},
            timeout=10
        )
        r.raise_for_status()
    except HTTPError as err:
        if err.response.status_code == 401:
            print(f"⚠ Skipping season {sid}: Unauthorized")
            continue
        else:
            raise
    fx = r.json()["data"].get("fixtures", [])
    print(f"→ Season {sid}: {len(fx)} fixtures")
    all_fx.extend(fx)

with sqlite3.connect(DB, timeout=30) as conn:
    cur = conn.cursor()
    for f in all_fx:
        cur.execute("""
          INSERT OR REPLACE INTO fixtures
            (id,season_id,league_id,date_time,status,home_team_id,away_team_id,home_score,away_score)
          VALUES (?,?,?,?,?,?,?,?,?);
        """, (
            f["id"], f.get("season_id"), f.get("league_id"),
            f.get("starting_at"), f.get("time",{}).get("status"),
            f.get("localteam_id"), f.get("visitorteam_id"),
            f.get("scores",{}).get("localteam_score"),
            f.get("scores",{}).get("visitorteam_score")
        ))
    conn.commit()

count = sqlite3.connect(DB).execute("SELECT COUNT(*) FROM fixtures").fetchone()[0]
print(f"Fixtures total: {count}")

→ Fetching fixtures for 133 seasons (~133 API calls)
→ Season 2: 380 fixtures
→ Season 3: 380 fixtures
→ Season 2052: 380 fixtures
→ Season 2053: 380 fixtures
→ Season 6: 380 fixtures
→ Season 7: 380 fixtures
→ Season 8: 380 fixtures
→ Season 2054: 380 fixtures
→ Season 2055: 380 fixtures
→ Season 15875: 62 fixtures
→ Season 15: 557 fixtures
→ Season 16: 555 fixtures
→ Season 17: 557 fixtures
→ Season 18: 557 fixtures
→ Season 19: 557 fixtures
→ Season 2065: 462 fixtures
→ Season 2066: 462 fixtures
→ Season 2067: 462 fixtures
→ Season 2068: 461 fixtures
→ Season 2078: 184 fixtures
→ Season 2079: 182 fixtures
→ Season 2080: 242 fixtures
→ Season 2081: 241 fixtures
→ Season 1574: 380 fixtures
→ Season 1575: 380 fixtures
→ Season 1576: 380 fixtures
→ Season 1577: 380 fixtures
→ Season 5671: 88 fixtures
→ Season 5672: 94 fixtures
→ Season 556: 305 fixtures
→ Season 557: 306 fixtures
→ Season 558: 305 fixtures
→ Season 559: 307 fixtures
→ Season 560: 289 fixtures
→ Season 5673: 94 fixtures


In [12]:
import sqlite3, requests, time
from requests.exceptions import HTTPError

API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0p2atkZnySe4X3xrHkjgGhOvI0pd"
FB_BASE   = "https://api.sportmonks.com/v3/football"
DB = "sportmonks.db"

def fetch_all_football(ep, params):
    items, page = [], 1
    while True:
        p = {"api_token": API_TOKEN, "per_page": 1000, "page": page, **params}
        r = requests.get(f"{FB_BASE}/{ep}", params=p, timeout=10)
        if r.status_code == 429:
            time.sleep(r.json()["rate_limit"]["resets_in_seconds"]+1)
            continue
        r.raise_for_status()
        data = r.json().get("data", [])
        if not data:
            break
        items.extend(data)
        if not r.json().get("pagination",{}).get("has_more", False):
            break
        page += 1
    return items

# 1) Build (team,season) pairs
pairs = set()
with sqlite3.connect(DB) as conn:
    for sid, home, away in conn.execute("SELECT season_id, home_team_id, away_team_id FROM fixtures"):
        if home: pairs.add((home, sid))
        if away: pairs.add((away, sid))

# 2) Pull squads
all_players = {}
for team_id, season_id in pairs:
    squad = fetch_all_football(
        "squads",
        {"filter[season_id]": season_id, "filter[team_id]": team_id, "include": "player"}
    )
    for e in squad:
        p = e.get("player")
        if p:
            p["team_id"] = team_id
            all_players[p["id"]] = p

players = list(all_players.values())
print(f"→ Collected {len(players)} unique players")

# 3) Recreate players table
with sqlite3.connect(DB, timeout=30) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS players;")
    cur.execute("""
      CREATE TABLE players (
        id            INTEGER PRIMARY KEY,
        firstname     TEXT,
        lastname      TEXT,
        fullname      TEXT,
        common_name   TEXT,
        nationality   TEXT,
        country_id    INTEGER,
        birthcountry  TEXT,
        birthdate     TEXT,
        position      TEXT,
        height        INTEGER,
        weight        INTEGER,
        image_path    TEXT,
        team_id       INTEGER
      );
    """)
    for p in players:
        cur.execute("""
          INSERT OR REPLACE INTO players 
          (id,firstname,lastname,fullname,common_name,nationality,
           country_id,birthcountry,birthdate,position,height,weight,image_path,team_id)
          VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?);
        """, (
            p["id"], p.get("firstname"), p.get("lastname"), p.get("fullname"),
            p.get("common_name"), p.get("nationality"), p.get("country_id"),
            p.get("birthcountry"), p.get("birthdate"), p.get("position"),
            p.get("height"), p.get("weight"), p.get("image_path"), p.get("team_id")
        ))
    conn.commit()

print(f"✔ Players table populated with {len(players)} records")


→ Collected 0 unique players
✔ Players table populated with 0 records


In [21]:
import sqlite3, requests, time
from requests.exceptions import HTTPError

API_TOKEN = "…"
BASE_URL = "https://api.sportmonks.com/v3/football"
DB = "sportmonks.db"

# 1) Last-5 seasons per league
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    cur.execute("SELECT id FROM leagues")
    season_ids = []
    for (lid,) in cur.fetchall():
        cur.execute("""
          SELECT id
            FROM seasons
           WHERE league_id=?
        ORDER BY start_date DESC
           LIMIT 5
        """, (lid,))
        season_ids += [r[0] for r in cur.fetchall()]
season_ids = list(set(season_ids))
print(f"→ Fetching fixtures for {len(season_ids)} seasons")

# 2) Recreate fixtures table
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS fixtures;")
    cur.execute("""
      CREATE TABLE fixtures (
        id            INTEGER PRIMARY KEY,
        season_id     INTEGER,
        league_id     INTEGER,
        date_time     TEXT,
        status        TEXT,
        home_team_id  INTEGER,
        away_team_id  INTEGER,
        home_score    INTEGER,
        away_score    INTEGER
      );
    """)
    conn.commit()

# 3) Fetch + insert
all_fx = []
for sid in season_ids:
    try:
        r = requests.get(
            f"{BASE_URL}/seasons/{sid}",
            params={"api_token": API_TOKEN, "include": "fixtures"},
            timeout=10
        )
        r.raise_for_status()
        fx = r.json().get("data", {}).get("fixtures", [])
    except HTTPError as e:
        fx = []
    print(f"Season {sid}: {len(fx)} fixtures")
    all_fx.extend(fx)

with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    for f in all_fx:
        cur.execute("""
          INSERT OR REPLACE INTO fixtures
            (id,season_id,league_id,date_time,status,
             home_team_id,away_team_id,home_score,away_score)
          VALUES (?,?,?,?,?,?,?,?,?);
        """, (
            f["id"], f.get("season_id"), f.get("league_id"),
            f.get("starting_at"), f.get("time", {}).get("status"),
            f.get("localteam_id"), f.get("visitorteam_id"),
            f.get("scores", {}).get("localteam_score"),
            f.get("scores", {}).get("visitorteam_score")
        ))
    conn.commit()

print("Fixtures total:", sqlite3.connect(DB).execute("SELECT COUNT(*) FROM fixtures").fetchone()[0])


→ Fetching fixtures for 133 seasons
Season 2: 0 fixtures
Season 3: 0 fixtures
Season 2052: 0 fixtures
Season 2053: 0 fixtures
Season 6: 0 fixtures
Season 7: 0 fixtures
Season 8: 0 fixtures
Season 2054: 0 fixtures
Season 2055: 0 fixtures
Season 15875: 0 fixtures
Season 15: 0 fixtures
Season 16: 0 fixtures
Season 17: 0 fixtures
Season 18: 0 fixtures
Season 19: 0 fixtures
Season 2065: 0 fixtures
Season 2066: 0 fixtures
Season 2067: 0 fixtures
Season 2068: 0 fixtures
Season 2078: 0 fixtures
Season 2079: 0 fixtures
Season 2080: 0 fixtures
Season 2081: 0 fixtures
Season 1574: 0 fixtures
Season 1575: 0 fixtures
Season 1576: 0 fixtures
Season 1577: 0 fixtures
Season 5671: 0 fixtures
Season 5672: 0 fixtures
Season 556: 0 fixtures
Season 557: 0 fixtures
Season 558: 0 fixtures
Season 559: 0 fixtures
Season 560: 0 fixtures
Season 5673: 0 fixtures
Season 5674: 0 fixtures
Season 1587: 0 fixtures
Season 1588: 0 fixtures
Season 1590: 0 fixtures
Season 1591: 0 fixtures
Season 2126: 0 fixtures
Season 21

In [29]:
import sqlite3
import requests
import time
from requests.exceptions import HTTPError

API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
BASE_URL  = "https://api.sportmonks.com/v3/football"
DB        = "sportmonks.db"

# 1) Grab all league IDs
with sqlite3.connect(DB) as conn:
    league_ids = [row[0] for row in conn.execute("SELECT id FROM leagues;")]

# 2) Recreate fixtures table
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS fixtures;")
    cur.execute("""
      CREATE TABLE fixtures (
        id            INTEGER PRIMARY KEY,
        league_id     INTEGER,
        date_time     TEXT,
        status        TEXT,
        home_team_id  INTEGER,
        away_team_id  INTEGER,
        home_score    INTEGER,
        away_score    INTEGER
      );
    """)
    conn.commit()

# 3) Fetch fixtures league-by-league
total = 0
for lid in league_ids:
    page = 1
    while True:
        resp = requests.get(
            f"{BASE_URL}/fixtures",
            params={
                "api_token":  API_TOKEN,
                "filter[league_id]": lid,
                "per_page":   1000,
                "page":       page
            },
            timeout=10
        )
        # back off on rate-limit
        if resp.status_code == 429:
            wait = resp.json().get("rate_limit",{}).get("resets_in_seconds",60)
            time.sleep(wait+1)
            continue
        resp.raise_for_status()
        data = resp.json().get("data", [])
        if not data:
            break

        with sqlite3.connect(DB) as conn:
            cur = conn.cursor()
            for f in data:
                cur.execute("""
                  INSERT OR REPLACE INTO fixtures
                    (id,league_id,date_time,status,
                     home_team_id,away_team_id,home_score,away_score)
                  VALUES (?,?,?,?,?,?,?,?);
                """, (
                    f["id"], lid,
                    f.get("starting_at"),
                    f.get("time",{}).get("status"),
                    f.get("localteam_id"),
                    f.get("visitorteam_id"),
                    f.get("scores",{}).get("localteam_score"),
                    f.get("scores",{}).get("visitorteam_score")
                ))
                total += 1
            conn.commit()

        print(f"→ League {lid}, page {page}: {len(data)} fixtures")
        page += 1

        # polite pause
        time.sleep(0.2)

print(f"✔ Loaded {total} total fixtures")

HTTPError: 400 Client Error: Bad Request for url: https://api.sportmonks.com/v3/football/fixtures?api_token=oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd&filter%5Bleague_id%5D=8&per_page=1000&page=1

In [24]:
import sqlite3

DB = "sportmonks.db"
with sqlite3.connect(DB) as conn:
    cur = conn.cursor()
    lg_cnt = cur.execute("SELECT COUNT(*) FROM leagues;").fetchone()[0]
    ss_cnt = cur.execute("SELECT COUNT(*) FROM seasons;").fetchone()[0]
    sample_leagues = cur.execute("SELECT id,name FROM leagues LIMIT 5;").fetchall()
    sample_seasons = cur.execute("SELECT id,league_id,start_date FROM seasons LIMIT 5;").fetchall()

print("Leagues count:   ", lg_cnt)
print("Seasons count:   ", ss_cnt)
print("Sample leagues:  ", sample_leagues)
print("Sample seasons:  ", sample_seasons)


Leagues count:    27
Seasons count:    531
Sample leagues:   [(8, 'Premier League'), (9, 'Championship'), (24, 'FA Cup'), (27, 'Carabao Cup'), (72, 'Eredivisie')]
Sample seasons:   [(2, 8, None), (3, 8, None), (6, 8, None), (7, 8, None), (8, 8, None)]


In [20]:
conn = sqlite3.connect('sportmonks.db')
cur = conn.cursor()
cur.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
for row in cur.fetchall():
    print(row)
conn.close()


In [15]:
import requests

API_TOKEN = "oYeoAVFUTQpu7MfoFqbvyiYfgRRkuBWW0p2atkZnySe4X3xrHkjgGhOvI0pd"
url = f"https://api.sportmonks.com/v3/core/continents"
resp = requests.get(url, params={"api_token": API_TOKEN, "per_page": 1})
print("Status code:", resp.status_code)
print("Sample response:", resp.json())


Status code: 200
Sample response: {'data': [{'id': 1, 'name': 'Europe', 'code': 'EU'}], 'pagination': {'count': 1, 'per_page': 1, 'current_page': 1, 'next_page': 'https://api.sportmonks.com/v3/core/continents?per_page=1&page=2', 'has_more': True}, 'subscription': [{'meta': {'trial_ends_at': '2025-04-27 17:59:05', 'ends_at': '2025-04-27 17:59:05', 'current_timestamp': 1745770144}, 'plans': [{'plan': 'European Plan', 'sport': 'Football', 'category': 'Standard'}], 'add_ons': [], 'widgets': []}], 'rate_limit': {'resets_in_seconds': 2386, 'remaining': 2994, 'requested_entity': 'Continent'}, 'timezone': 'UTC'}
